In [1]:
# 增强锐度 有用
import time
from PIL import Image,ImageFilter,ImageEnhance
import os
from PIL import ImageGrab

img = Image.open('H:\\edge_dection\\Jockey_001\\HR\\001.png')
sharpness = ImageEnhance.Sharpness(img)
img = sharpness.enhance(10)
img.save(os.path.join('H:\\edge_dection\\Jockey_001\\sharp\\001.png'))



FileNotFoundError: [Errno 2] No such file or directory: 'H:\\edge_dection\\Jockey_001\\HR\\001.png'

In [13]:
# 增强锐度 有用
import time
from PIL import Image,ImageFilter,ImageEnhance
import os
from PIL import ImageGrab

#  源目录
input_path = 'H:\\edge_dection\\Jockey_001\\HR\\'
#  输出目录
output_path = 'H:\\edge_dection\\Jockey_001\\sharp\\'

def imageResize(input_path, output_path):
    # 获取输入文件夹中的所有文件/夹，并改变工作空间
    files = os.listdir(input_path)
    os.chdir(input_path)
    # 判断输出文件夹是否存在，不存在则创建
    if (not os.path.exists(output_path)):
        os.makedirs(output_path)
    for file in files:
        # 判断是否为文件，文件夹不操作
        if (os.path.isfile(file)):
            img = Image.open(file)
            # 增强锐度
            start = time.time()
            sharpness = ImageEnhance.Sharpness(img)
            img = sharpness.enhance(10)
            end = time.time()
            print("循环运行时间:%.2f秒"%(end-start))
            img.save(os.path.join(output_path, file))
imageResize(input_path, output_path)

'''Jockey_000
最小平均增强锐度时间=0.43s
sharp+轮廓cut=1.06s or 2.69s
'''
'''Jockey_008
最小平均增强锐度时间=0.4s
sharp+轮廓cut=0.69s or 2.4s
'''
'''Ready_000
最小平均增强锐度时间=0.5s
sharp+轮廓cut=1.01s or 3.31s
'''
'''Ready_008
最小平均增强锐度时间=0.42s
sharp+轮廓cut=0.80s or 3.01s
'''

循环运行时间:0.35秒


'Ready_008\n最小平均增强锐度时间=0.42s\nsharp+轮廓cut=0.80s or 3.01s\n'

In [4]:
# 增强锐度+随机裁剪 有用
import time
from PIL import Image,ImageFilter,ImageEnhance
import os
from PIL import ImageGrab
import numpy as np
import cv2
import pandas as pd

def imageResize(input_path, output_path):
    # 获取输入文件夹中的所有文件/夹，并改变工作空间
    files = os.listdir(input_path)
    os.chdir(input_path)
    # 判断输出文件夹是否存在，不存在则创建
    if (not os.path.exists(output_path)):
        os.makedirs(output_path)
    for file in files:
        # 判断是否为文件，文件夹不操作
        if (os.path.isfile(file)):
            imgH = Image.open(file)
            #print(img.size)
            # 增强锐度
            sharpness = ImageEnhance.Sharpness(imgH)
            img = sharpness.enhance(10)
            img.save(os.path.join(output_path, file))
            return img,imgH


def get_loc(img, patch_size):#########随机补丁位置
    ih, iw = img.shape[:2]
    print("分辨率："+str(ih)+"x"+str(iw))
    #print(ih,iw)
    ip = patch_size
    C=[]
    R=[]
    L=[]
    for ix in range(0, iw - ip + 1,200):
        for iy in range(0,ih - ip + 1,200):
            #print(ix,iy)
            ret =np.array( 
                img[iy:iy + ip, ix:ix + ip]
            )   #隔200取一个，分块
            #print(ret.shape)
            #cv2.imwrite('H:/edge_dection/Ready_008/'+str(ix)+"_"+str(iy)+'.png',ret )
            
            contours, hierarchy = cv2.findContours(ret, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#查找检测物体的轮廓
            R.append([ix,iy])#保存每个块起始位置
            #print(len(contours))
            C.append(len(contours))#保存每个块轮廓数
            
    '''取最大轮廓数索引，找最大轮廓位置，保存前10个块'''
    print("总块数：",len(C))
    num = len(C)
    print("提取块数：",int(len(C)))
    for i in range(int(len(C))):###########10 patch        
        print("最大轮廓数：",max(C))####
        index=C.index(max(C))#####
        print("最大值索引：",index)
        print("位置：      ",R[index])
        L.append(R[index])
        del C[index]
        del R[index]
    return num,L

def get_patch(name,imgH,imgL, i,ix,iy,scale,patch_size):#########轮廓 Top max
    #start = time.time()
    ip = patch_size
    
    '''scale=8,取HR、LR对应位置块'''
    retH =np.array( 
        imgH[iy:iy + ip, ix:ix + ip,:]
    )   #hr patch
    print(iy/scale,ix/scale)
    print("裁剪位置："+str(iy)+","+str(iy + ip)+","+str( ix)+","+str(ix + ip))
    retL =np.array( 
        imgL[int(iy/scale):int(iy/scale) + int(ip/scale), int(ix/scale):int(ix/scale) + int(ip/scale),:]
    )   #lr patch
    #end = time.time()
    #t=end-start
    #get_t.append(t)
    
    if i+1 <10:
        cv2.imwrite('H:/edge_dection/'+str(name)+'/sharp_patch_l/00'+str(i+1)+'.png',retH )
        cv2.imwrite('H:/edge_dection/'+str(name)+'/LR_patch_l/00'+str(i+1)+'.png',retL )
    else:
        cv2.imwrite('H:/edge_dection/'+str(name)+'/sharp_patch_l/0'+str(i+1)+'.png',retH )
        cv2.imwrite('H:/edge_dection/'+str(name)+'/LR_patch_l/0'+str(i+1)+'.png',retL )
        
def save_loc(name,L):
    # list转dataframe
    df = pd.DataFrame(L, columns=list('xy'))
    
    # 保存到本地excel
    df.to_excel("H:/edge_dection/"+str(name)+"_counter_loc.xlsx", index=False)

name = 'Jockey_001'#####
scale = 8 ######3
patch_size = 400

#  源目录
input_path = 'H:\\edge_dection\\'+str(name)+'\\HR\\'#'C:\\Users\\aqing\\Desktop\\j001_yu\\SR'

#  输出目录
output_path = 'H:\\edge_dection\\'+str(name)+'\\sharp\\'

start = time.time()
#1.
img,imgH = imageResize(input_path, output_path)
imgH = np.asarray(imgH.convert('L'))
imgH= cv2.Canny(imgH,10,100)
#img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,401,-10)

'''
cv2.namedWindow('original', cv2.WINDOW_NORMAL)
cv2.resizeWindow("original", 800, 500) 
cv2.imshow("original", imgH) 
cv2.waitKey(0)
cv2.destroyAllWindows()
#print(img.size)
'''

#2.
num,L= get_loc(np.array(imgH),patch_size)#10个块位置
#裁切位置
save_loc(name,L)
#'''
#3.
img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
print(img.size)
#imgH = cv2.imread('H:/edge_dection/Ready_008/009.png')
imgL = cv2.imread('H:/edge_dection//'+str(name)+'/LR_bicubic/'+str(scale)+'x/'+'001.png')
##imgL = cv2.imread('H:/edge_dection/'+str(name)+'/LR_bicubic/'+str(scale)+'x/'+str(name)+'_bic.png')####
#print(imgL)
for i in range(num):###########10 patch ；i,j变
    ix = L[i][0]
    iy = L[i][1]
    #get_patch(name,img,imgL,i,ix,iy,scale,patch_size)
    
end = time.time()
print("循环运行时间:%.2f秒"%(end-start))
#'''
'''
Jockey_000 2.08秒 0.60秒 
Jockey_008 2.03秒 0.55秒
Ready_000  2.46秒 1.00秒
Ready_008  2.15秒 0.61秒
'''

分辨率：2160x3840
总块数： 162
提取块数： 162
最大轮廓数： 254
最大值索引： 119
位置：       [2600, 400]
最大轮廓数： 214
最大值索引： 127
位置：       [2800, 400]
最大轮廓数： 213
最大值索引： 109
位置：       [2400, 200]
最大轮廓数： 205
最大值索引： 126
位置：       [2800, 600]
最大轮廓数： 197
最大值索引： 109
位置：       [2400, 400]
最大轮廓数： 197
最大值索引： 117
位置：       [2600, 600]
最大轮廓数： 177
最大值索引： 116
位置：       [2600, 200]
最大轮廓数： 150
最大值索引： 65
位置：       [1400, 400]
最大轮廓数： 144
最大值索引： 99
位置：       [2200, 200]
最大轮廓数： 140
最大值索引： 99
位置：       [2200, 400]
最大轮廓数： 132
最大值索引： 65
位置：       [1400, 600]
最大轮廓数： 129
最大值索引： 127
位置：       [3000, 600]
最大轮廓数： 125
最大值索引： 72
位置：       [1600, 400]
最大轮廓数： 122
最大值索引： 125
位置：       [3000, 400]
最大轮廓数： 122
最大值索引： 133
位置：       [3200, 600]
最大轮廓数： 120
最大值索引： 72
位置：       [1600, 600]
最大轮廓数： 119
最大值索引： 21
位置：       [400, 600]
最大轮廓数： 118
最大值索引： 139
位置：       [3400, 600]
最大轮廓数： 115
最大值索引： 88
位置：       [2000, 600]
最大轮廓数： 113
最大值索引： 101
位置：       [2400, 600]
最大轮廓数： 112
最大值索引： 100
位置：       [2400, 0]
最大轮廓数： 109
最大值索引： 79
位置：       [1800, 600]
最大轮廓数： 109


'\nJockey_000 2.08秒 0.60秒 \nJockey_008 2.03秒 0.55秒\nReady_000  2.46秒 1.00秒\nReady_008  2.15秒 0.61秒\n'

In [8]:
# 增强锐度 有用

from PIL import Image,ImageFilter,ImageEnhance
import os
from PIL import ImageGrab
#  源目录
input_path = 'C:\\Users\\aqing\\Desktop\\j001_re\\SR'
#  输出目录
output_path = 'C:\\Users\\aqing\\Desktop\\j001_re'

def imageResize(input_path, output_path):
    # 获取输入文件夹中的所有文件/夹，并改变工作空间
    files = os.listdir(input_path)
    os.chdir(input_path)
    # 判断输出文件夹是否存在，不存在则创建
    if (not os.path.exists(output_path)):
        os.makedirs(output_path)
    for file in files:
        # 判断是否为文件，文件夹不操作
        if (os.path.isfile(file)):
            img = Image.open(file)
            # 增强锐度
            sharpness = ImageEnhance.Sharpness(img)
            img = sharpness.enhance(10)

            img.save(os.path.join(output_path, file))
imageResize(input_path, output_path)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'C:\\Users\\aqing\\Desktop\\j001_re\\SR'